# 第2部 因果探索
## 第6章 LiNGAMの実装

## 6-1 LiNGAM(Linear Non-Gaussian Acyclic Model)とは

#### ・因果探索 : 多くの項目をアンケート調査などで収集した後に、調査項目の間にある因果関係(ネットワークの繋がり方)を求めること .

#### 例として、「生活習慣と疾病の大規模調査」、「企業における働き方改革に伴う社員の意識調査」などが挙げられる .

#### ・本章では因果探索の代表的手法であるLiNGAM(Linear Non-Gaussian Acyclic Model)を解説、実装する .

## LiNGAMの前提

#### ・LiNGAMは構造方程式モデルを前提において、因果探索を実施する .

#### ・LiNGAM : 因果構造がDAG(有向非巡回グラフ)であり、誤差項が非ガウス分布である線形モデルのこと .

#### ・例えば、変数$x$,$y$,$z$の因果関係を構造方程式モデルで記載すると、

$$
\begin{aligned}
x &= f_x(y, z, e_x) \\
y &= f_y(x, z, e_y) \\
z &= f_z(x, y, e_z)
\end{aligned}
$$

#### となる .　LiNGAMでは、

$$
\begin{aligned}
x_1 &= b_{12}x_2 + b_{13}x_3 + e_1 \\
x_2 &= b_{21}x_1 + b_{23}x_3 + e_2 \\
x_3 &= b_{31}x_1 + b_{32}x_2 + e_3
\end{aligned}
$$

#### のように、線形な構造方程式を仮定する .　ここで、変数$x$,$y$,$z$を、変数$x_1$,$x_2$,$x_3$に書き直している .　観測したデータ$N$個をまとめて表すために、各変数$x_1$,$x_2$,$x_3$がベクトルで記載されている .

#### この式を、行列を使って書き直すと、

$$
\begin{pmatrix}
x_1 \\
x_2 \\
x_3
\end{pmatrix}
=
\begin{pmatrix}
0 & b_{12} & b_{13} \\
b_{21} & 0 & b_{23} \\
b_{31} & b_{32} & 0
\end{pmatrix}
\begin{pmatrix}
x_1 \\
x_2 \\
x_3
\end{pmatrix}
+
\begin{pmatrix}
e_1 \\
e_2 \\
e_3
\end{pmatrix}
$$

#### となる .　各ベクトルと行列をそれぞれ$\mathbf{x}$,$B$,$\mathbf{e}$で表すと、次のようになる .

$$
\mathbf{x} = B\mathbf{x} + \mathbf{e}
$$

#### ・LiNGAMによる因果探索とは、観測されているデータ$\mathbf{x} = (x_1, x_2, x_3)^{\mathrm{T}}$から$b_{12}$や$b_{13}$の値を推定し、行列$B$を求め、線形な構造方程式を求めることになる .

## 非循環性(Acyclic)による行列$B$の制限

#### ・LiNGAMは非循環を前提とするので、行列$B$には制限がかかる .

#### 前出の数式では、自分への循環がないので、対角成分は0となる .

#### ・さらに、変数が循環することがないので、因果の上流にある変数から順番を記載するように変数$x_1$,$x_2$,$x_3$の順番を並び替えれば、行列$B$は下三角行列になる .

#### 例えば、変数$x_1$,$x_2$,$x_3$を因果の上流から並び変えると、変数$x_2$,$x_1$,$x_3$となる場合、行列で表した式は、

$$
\begin{pmatrix}
x_2 \\
x_1 \\
x_3
\end{pmatrix}
=
\begin{pmatrix}
0 & 0 & 0 \\
b_{12} & 0 & 0 \\
b_{31} & b_{32} & 0
\end{pmatrix}
\begin{pmatrix}
x_2 \\
x_1 \\
x_3
\end{pmatrix}
+
\begin{pmatrix}
e_2 \\
e_1 \\
e_3
\end{pmatrix}
$$

#### となる .　この行列から言えることは、以下の2つである .

#### ・最上流の変数$x_2$には、下流の変数$x_1$,$x_3$からは影響がない .
#### ・2番目の変数$x_1$の場合は、その上流にある変数$x_2$からは影響があるが、下流にある変数$x_3$からは因果がなく、影響がない .

#### 因果の上流から並び替えた状態で変数$x_1$,$x_2$,$x_3$を割り当て直して記載する .　つまり、

$$
\begin{pmatrix}
x_1 \\
x_2 \\
x_3
\end{pmatrix}
=
\begin{pmatrix}
0 & 0 & 0 \\
b_{21} & 0 & 0 \\
b_{31} & b_{32} & 0
\end{pmatrix}
\begin{pmatrix}
x_1 \\
x_2 \\
x_3
\end{pmatrix}
+
\begin{pmatrix}
e_1 \\
e_2 \\
e_3
\end{pmatrix}
$$

#### となる .　これは、$\mathbf{x} = B\mathbf{x} + \mathbf{e}$と表され、行列$B$は下三角行列です .

#### ・対角成分より上側の項が0でない場合、すなわち$b_{ij}≠0$(ただし、$i<j$)のとき、変数$x_j$から変数$x_i$への因果の矢印があることになる .

#### 行列$B$の下側は成分を持つので$b_{ji}≠0$である .　よって、対角成分より上側の項が0でない場合、$b_{ij}≠0$であり$b_{ji}≠0$でもあるので、変数$x_i$と変数$x_j$で双方向に循環が生まれる($x_j$→$x_i$→$x_j$) .

#### これでは非循環の前提が成り立たない .　非循環な構造方程式モデルでは変数$x_1$,$x_2$,$x_3$の順番を並び替えることで、行列$B$は下三角行列になる .

## 行列$B$の求め方

#### ・LiNGAMでは行列$B$が求まれば、構造方程式モデル、そして因果ダイアグラムが明らかになる .　構造方程式モデルを変形させて、

$$
\mathbf{x} = B\mathbf{x} + \mathbf{e} \\
(I - B)\mathbf{x}= \mathbf{e} \\
\mathbf{x} = (I - B)^{-1} \mathbf{e}
$$

#### ここで、$I$は単位行列を示している .　$(I - B)^{-1}$を$A$と記載すると、$\mathbf{x}=A\mathbf{e}$と変形でき、この$A$を求めればよいことが分かる .

## 6-2 独立成分分析(ICA : Independent Component Analysis)

#### ・機械学習の教師なし学習手法に、データの次元圧縮で使用する主成分分析(PCA : Principal Component Analysis)と呼ばれる手法がある .
####  これはデータ$\mathbf{x}$を、各要素間で"相関が0になるように変換した"、$\mathbf{x}_{pca}$という新たな変数に線形変換する操作のこと .

#### ・独立成分分析は主成分分析をさらに発展させた手法であり、相関が0になった$\mathbf{x}_{pca}$をさらに線形変換して、要素間の関係を独立にする操作である .

#### ・データ$\mathbf{x}$がガウス分布に従う場合に独立と無相関は同じ状況を指す .　一方で対象データが非ガウス分布に従う場合、主成分分析で無相関になるが、独立にはならない(独立→無相関、無相関$\not\rightarrow$独立) .

#### ・独立成分分析は、主成分分析の結果$\mathbf{x}_{pca}$に対し、データが非ガウスであるという特徴を使って線形変換を実施し、$\mathbf{x}_{pca}$の要素間が独立となる新たな変数$\mathbf{x}_{ica}$を作成する .　(具体的な手法はここでは省略し、個人の学習とする .)

#### 独立成分分析は機械学習ライブラリscikit-learnに搭載されているので、簡単に使用できる .

#### ・本書で押さえておきたい内容は、独立成分分析を利用することで、データ$\mathbf{x}$を

$$
\mathbf{x} = A_{ica}x_{ica}
$$

#### と、行列$A_{ica}$と$\mathbf{x}_{ica}$に分解できるということである .　ここで、$\mathbf{x}_{ica}$を$\mathbf{e}_{ica}$と記載し直すと、

$$
\mathbf{x} = A_{ica}\mathbf{e}_{ica}
$$

#### である .　独立成分分析の対象が非ガウスという仮定は、LiNGAMで想定していた前提「ノイズがnon-Gaussian」とも一致する .　前節の最後で紹介した$(I - B)^{-1}=A$としたときのLiNGAMの数式は、

$$
\mathbf{x}=A\mathbf{e}
$$

#### であり、$\mathbf{e}$が非ガウスなノイズであった .　独立成分分析による変数変換と、LiNGAMで求めたい式の形が一致していることがわかる .

#### よって、LiNGAMでは観測したデータ$x$に対して、$\mathbf{x}=A\mathbf{e}$の独立成分分析を実施することで構造方程式を求めることができる .

## 独立成分分析とLiNGAMをつなぐ

#### ・独立成分分析によりデータ$\mathbf{x}$を

$$
\mathbf{x} = A_{ica}\mathbf{e}_{ica}
$$

#### と分解するような行列$A_{ica}$と$\mathbf{e}_{ica}$が求まる .

#### ・しかし、この行列$A_{ica}$は、LiNGAMで求めたい構造方程式モデル

$$
\mathbf{x} = B\mathbf{x} + \mathbf{e}
$$

#### において、$(I - B)^{-1}=A$としたときの

$$
\mathbf{x}=A\mathbf{e}
$$

#### の$A$とは必ずしも一致しない .

#### ・独立成分分析における行列$A_{ica}$と$e_{ica}$の特徴について、$A_{ica}$には"倍数"と"行交換"の自由度がある .　すなわち、$A_{ica}$を適当にD倍した場合、$e_{ica}$が$1/D$になれば良いので、$A_{ica}$の各要素において大きさが不定である .

#### ・また、$e_{ica}$が$e_1$,$e_2$,$e_3$と並んでいる場合でも、順番を適当に変えて、$e_3$,$e_1$,$e_2$と並んでいる場合でも$A_{ica}$の要素の配置を変えれば元のデータ$\mathbf{x}$が復元できるので、行交換に対しても不定である .

#### ・次に、LiNGAMにおける行列$A$の特徴を整理する .　LiNGAMではデータ$x$の要素の順番を因果の上流から並べて記載し、非巡回である .

#### これらの前提から係数行列$B$は下三角行列となった .

#### そのため、LiNGAMにおいて$A=(I - B)^{-1}$の逆行列である$A^{-1}=(I-B)$を考えたときには、係数行列$B$が下三角行列であることから、$A^{-1}$は対角成分が1、そして、対角成分より上側はすべて要素が0、という特徴を持つ .

#### ・ゆえに、$A^{-1}_{ica}$は対角成分が1、そして、対角成分より上側はすべて要素が0という特徴となるように、"行の大きさを調整"、そして"行の順番を交換"してあげる必要がある .
#### これらの操作を行うことで、LiNGAMで因果関係を求めるための係数行列$B$を求めることができる .

## 6-3 LiNGAMによる因果探索の実装

#### ・疑似データを作成して、LiNGAMによる因果探索を実際に実装する .

## 因果探索に使用する疑似データの構造

#### ・因果探索に使用する疑似データを作る .　疑似データを、構造方程式モデルで記載すると、

$$
x_1 = 3 \times x_2 + e_{x_1} \\
x_2 = e_{x_2} \\
x_3 = 2 \times x_1 + 4 \times x_2 + e_{x_3}
$$

#### とする .　因果ダイアグラムで記載すると以下の図6.3.1となる(図の変数$Y_3$は誤りで、正しくは$x_3$) .

![alt text](pict1.png)

#### まず、これらのデータを生成する .

### プログラム実行前の設定など

In [16]:
# 乱数のシードを固定
import random
import numpy as np

random.seed(1234)
np.random.seed(1234)

In [17]:
# 使用するパッケージ（ライブラリと関数）を定義
import pandas as pd

### データの生成

In [18]:
# データ数
num_data = 200

# 非ガウスのノイズ
ex1 = 2*(np.random.rand(num_data)-0.5)  # -1.0から1.0
ex2 = 2*(np.random.rand(num_data)-0.5)
ex3 = 2*(np.random.rand(num_data)-0.5)

# データ生成
x2 = ex2
x1 = 3*x2 + ex1
x3 = 2*x1 + 4*x2 + ex3

# 表にまとめる
df = pd.DataFrame({"x1": x1, "x2": x2, "x3": x3})
df.head()

,x1,x2,x3
0,2.257272,0.958078,8.776842
1,2.531611,0.762464,8.561263
2,0.641547,0.255364,1.341902
3,3.153636,0.860973,9.322791
4,1.908691,0.449580,5.776675


#### ・それではLiNGAMを実施する .　最初は独立成分分析を行い、$A^{-1}_{ica}$を求める .

### 独立成分分析を実施

In [19]:
# 独立成分分析はscikit-learnの関数を使用します
from sklearn.decomposition import FastICA
# https://scikit-learn.org/stable/modules/generated/sklearn.decomposition.FastICA.html

ica = FastICA(random_state=1234, max_iter=1000).fit(df)

# ICAで求めた行列A
A_ica = ica.mixing_

# 行列Aの逆行列を求める
A_ica_inv = np.linalg.pinv(A_ica)
print(A_ica_inv)

[[-0.30765228  1.85303772  0.0776654 ]
 [-1.60473574  5.7060046  -0.07796094]
 [-3.2797495  -6.56171813  1.63292454]]


#### ・次に、$A^{-1}_{ica}$に対して、①「行の順番を変換」、②「行の大きさを調整」し、対角成分が1、そして、対角成分より上側はすべて要素が0となるようにする .

### munkresのインポート

In [20]:
!pip install munkres
from munkres import Munkres
from copy import deepcopy

#### ・次に、$A^{-1}_{ica}$に対して、①「行の順番を変更」を行う .

#### 行の順番を並び替える基準について、求めたい$A^{-1}_{ica}$は対角成分が1になるものであった .　すなわち対角成分が非ゼロである .　よって、"$A^{-1}_{ica}$の対角成分の絶対値をできるだけ大きくしたい"ということになる .

#### "$A^{-1}_{ica}$の対角成分の絶対値をできるだけ大きくしたい"という行為は、"$A^{-1}_{ica}$の絶対値行列を逆数にした行列において、対角成分の和が最小になるようにしたい"という行為になる .

In [21]:
# 実装の参考
# [5] Qiita：LiNGAMモデルの推定方法について
# https://qiita.com/m__k/items/bd87c063a7496897ba7c

# ①「行の順番を変換」→対角成分の絶対値を最大にする
# （元のA^-1の対角成分は必ず0ではないので）

# 絶対値の逆数にして対角成分の和を最小にする問題に置き換える
A_ica_inv_small = 1 / np.abs(A_ica_inv)

# 対角成分の和を最小にする行の入れ替え順を求める
m = Munkres()  # ハンガリアン法
ixs = np.vstack(m.compute(deepcopy(A_ica_inv_small)))

# 求めた順番で変換
ixs = ixs[np.argsort(ixs[:, 0]), :]
ixs_perm = ixs[:, 1]
A_ica_inv_perm = np.zeros_like(A_ica_inv)
A_ica_inv_perm[ixs_perm] = A_ica_inv
print(A_ica_inv_perm)

[[-1.60473574  5.7060046  -0.07796094]
 [-0.30765228  1.85303772  0.0776654 ]
 [-3.2797495  -6.56171813  1.63292454]]


In [22]:
# 並び替わった順番
print(ixs)

[[0 1]
 [1 0]
 [2 2]]


#### ・この出力は、元のindex0(1行目)はindex1(2行目)になり、2行目は1行目になり、3行目は変化なしであるということを示している .

#### ・次に、$A^{-1}_{ica}$の各行の大きさを調整する .　ここでは対角成分が1になるように各行を割り算する .


In [24]:
# ②「行の大きさを調整」
D = np.diag(A_ica_inv_perm)[:, np.newaxis]  # D倍されているDを求める
A_ica_inv_perm_D = A_ica_inv_perm / D
print(A_ica_inv_perm_D)

[[ 1.         -3.55572849  0.04858179]
 [-0.16602591  1.          0.04191247]
 [-2.0085126  -4.01838418  1.        ]]


#### ・次に、$A^{-1}=(I-B)$なので、$B=I-A^{-1}$として$B$を計算する .

In [25]:
# ③「B=I-A_inv」
B_est = np.eye(3) - A_ica_inv_perm_D
print(B_est)

[[ 0.          3.55572849 -0.04858179]
 [ 0.16602591  0.         -0.04191247]
 [ 2.0085126   4.01838418  0.        ]]


#### ・出力された結果から、推定した$B_{ets}$は下三角行列ではない .　そもそも今回は変数が因果の上流から順番には並んでおらず、

$$
x_1 = 3 \times x_2 + e_{x_1} \\
x_2 = e_{x_2} \\
x_3 = 2 \times x_1 + 4 \times x_2 + e_{x_3}
$$

#### より、正解の$B$は

$$
B = 
\begin{pmatrix}
0 & 3 & 0 \\
0 & 0 & 0 \\
2 & 4 & 0
\end{pmatrix}
$$

#### でした .　$B_{est}$には計算の誤差、そしてデータが有限なため、0になるべき部分が完璧には0になっていない .　また今回はどこが0になるのか分かっているから良いが、未知のデータでは「何個が0になるのか？」、「どの要素が0になるべきなのか？」は不明である .

#### ・そこで手続きとしては、

#### ①上側成分の0になるはずの数(3×3であれば3個、4×4であれば6個と、対角成分の上側の要素数分)、絶対値が小さい成分を0にする .

#### ②そして、変数の順番を入れ替えて、$B_{est-transformed}$が下三角行列になるかを確かめる

#### を行う .　これでまだ$B_{est-transformed}$が下三角行列でない場合は、さらに1つ絶対値の小さな成分を0にして、②を実施、を繰り返す .

#### ・まず$B_{est}$に①上側成分の0になるはずの数(今回は3個)分、絶対値が小さい成分を0にするを実施すると、

$$
B_{\text{est}} =
\begin{pmatrix}
0 & 3.57021564 & 0 \\
0 & 0 & 0 \\
2.00970483 & 4.01538182 & 0
\end{pmatrix}
$$

#### となる .　そして、これは、②変数の順番を入れ替えて、$B_{est-transformed}$が下三角行列になるかを確かめる .　変数の1番目と2番目を入れ替えた場合を考える .　このときは、

$$
\begin{pmatrix}
x_1 \\
x_2 \\
x_3
\end{pmatrix}
=
\begin{pmatrix}
0 & b_{12} & b_{13} \\
b_{21} & 0 & b_{23} \\
b_{31} & b_{32} & 0
\end{pmatrix}
\begin{pmatrix}
x_1 \\
x_2 \\
x_3
\end{pmatrix}
+
\begin{pmatrix}
e_1 \\
e_2 \\
e_3
\end{pmatrix}
$$

#### が、

$$
\begin{pmatrix}
x_2 \\
x_1 \\
x_3
\end{pmatrix}
=
\begin{pmatrix}
0 & b_{21} & b_{23} \\
b_{12} & 0 & b_{13} \\
b_{32} & b_{31} & 0
\end{pmatrix}
\begin{pmatrix}
x_2 \\
x_1 \\
x_3
\end{pmatrix}
+
\begin{pmatrix}
e_2 \\
e_1 \\
e_3
\end{pmatrix}
$$

#### となるため、

$$
B_{\text{est.transformed}} =
\begin{pmatrix}
0 & 0 & 0 \\
3.57021564 & 0 & 0 \\
4.01538182 & 2.00970483 & 0
\end{pmatrix}
$$

#### となり、下三角行列となる .

#### ・よって、$B_{est}$は非循環の条件を満たしている行列となっていることが分かる .　実装は以下の通りである .

In [26]:
# ①上側成分の0になるはずの数（3×3であれば3個、4×4であれば6個と、対角成分の上側の要素数分）、絶対値が小さい成分を0にする
# ②変数の順番を入れ替えて、下三角行列になるかを確かめる、
# 実装の参考
# [5] Qiita：LiNGAMモデルの推定方法について
# https://qiita.com/m__k/items/bd87c063a7496897ba7c

def _slttestperm(b_i):
# b_iの行を並び替えて下三角行列にできるかどうかチェック
    n = b_i.shape[0]
    remnodes = np.arange(n)
    b_rem = deepcopy(b_i)
    p = list() 

    for i in range(n):
        # 成分が全て0である行番号のリスト
        ixs = np.where(np.sum(np.abs(b_rem), axis=1) < 1e-12)[0]

        if len(ixs) == 0:
            return None
        else:
            ix = ixs[0]
            p.append(remnodes[ix])

            #　成分が全て0である行を削除
            remnodes = np.hstack((remnodes[:ix], remnodes[(ix + 1):]))
            ixs = np.hstack((np.arange(ix), np.arange(ix + 1, len(b_rem))))
            b_rem = b_rem[ixs, :]
            b_rem = b_rem[:, ixs]

    return np.array(p)

b = B_est
n = b.shape[0]
assert(b.shape == (n, n))

ixs = np.argsort(np.abs(b).ravel())

for i in range(int(n * (n + 1) / 2) - 1, (n * n) - 1):
    b_i = deepcopy(b)
    b_i.ravel()[ixs[:i]] = 0
    ixs_perm = _slttestperm(b_i)
    if ixs_perm is not None:
        b_opt = deepcopy(b)
        b_opt = b_opt[ixs_perm, :]
        b_opt = b_opt[:, ixs_perm]
        break
b_csl = np.tril(b_opt, -1)
b_csl[ixs_perm, :] = deepcopy(b_csl)
b_csl[:, ixs_perm] = deepcopy(b_csl)

B_est1 = b_csl
print(B_est1)

[[0.         3.55572849 0.        ]
 [0.         0.         0.        ]
 [2.0085126  4.01838418 0.        ]]


#### ・最後に再度、$B_{est}$の要素を回帰分析で求める .

#### 回帰分析で行列$B$の成分を再度求めるのは、現状の$B_{est}$の0でない要素が、絶対値の小さい成分のゼロ化の操作を実施する前の値であるため、ゼロ化操作後での係数の値を求めたいからである .

#### ・今回は$B_{est}$は3つの要素が0でないので、3つのパスが存在する .　パスが存在する部分のみを考慮した回帰モデルを構築し、因果の大きさを求める .

### 行列$B$の非ゼロ要素を求め直す

In [27]:
# scikit-learnから線形回帰をimport
from sklearn.linear_model import LinearRegression

# 説明変数
X1 = df[["x2"]]
X3 = df[["x1", "x2"]]

# 被説明変数（目的変数）
# df["x1"]
# df["x3"]

# 回帰の実施
reg1 = LinearRegression().fit(X1, df["x1"])
reg3 = LinearRegression().fit(X3, df["x3"])

# 回帰した結果の係数を出力
print("係数：", reg1.coef_)
print("係数：", reg3.coef_)

係数： [3.14642595]
係数： [1.96164568 4.11256441]


#### ・疑似データは以下の式、

$$
\begin{aligned}
x_1 &= 3 \times x_2 + e_{x_1} \\
x_2 &= e_{x_2} \\
x_3 &= 2 \times x_1 + 4 \times x_2 + e_{x_3}
\end{aligned}
$$

#### から生成したので、推定結果の

$$
\begin{aligned}
x_1 &= 3.1 \times x_2 + e_{x_1} \\
x_2 &= e_{x_2} \\
x_3 &= 2.0 \times x_1 + 4.1 \times x_2 + e_{x_3}
\end{aligned}
$$

#### は、元の構造方程式モデルとほぼ同じとなっている .　よって、観測したデータ$(x_1, x_2, x_3)^{\mathrm{T}}$から構造方程式を求めることができた .

## 次回

#### ・第7章では、ベイジアンネットワークと呼ばれる手法による因果探索について解説する .